In [1]:
import duckdb
import pandas as pd
import os

PAI_DIR = os.getcwd()  # diretório atual para notebooks jupyter
DATA_RAW = os.path.join(PAI_DIR, "data", "raw")
DATA_PROCESSED = os.path.join(PAI_DIR, "data", "processed")

In [2]:
con = duckdb.connect(database=':memory:')

In [3]:
for arquivo in os.listdir(DATA_PROCESSED):
    if arquivo.startswith("estimativa_cre_") and arquivo.endswith(".parquet"):
        caminho = os.path.join(DATA_PROCESSED, arquivo)

        # Nome da tabela = nome do arquivo sem extensão
        nome_tabela = os.path.splitext(arquivo)[0]

        # Cria tabela no DuckDB (nome da tabela entre aspas duplas para aceitar hífens)
        con.execute(f"""
            CREATE TABLE IF NOT EXISTS "{nome_tabela}" AS 
            SELECT * FROM '{caminho}'
            """)
        print(f"✅ Tabela {nome_tabela} criada a partir de {arquivo}")

✅ Tabela estimativa_cre_12-08-25 criada a partir de estimativa_cre_12-08-25.parquet
✅ Tabela estimativa_cre_16-06-25 criada a partir de estimativa_cre_16-06-25.parquet


In [4]:
tabelas = [
    "STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA.parquet",
    "STG_SIGEF__DIM__ACAO_PROGRAMA.parquet",
    "STG_SIGEF__DIM__CONTA_CONTABIL.parquet",
    "STG_SIGEF__DIM__CREDOR.parquet",
    "STG_SIGEF__DIM__DOMICILIO_BANCARIO.parquet",
    "STG_SIGEF__DIM__EVENTO.parquet",
    "STG_SIGEF__DIM__FONTE_RECURSO.parquet",
    "STG_SIGEF__DIM__GRUPO_PROGRAMACAO_FINANCEIRA.parquet",
    "STG_SIGEF__DIM__NATUREZA_DESPESA.parquet",
    "STG_SIGEF__DIM__NATUREZA_RECEITA.parquet",
    "STG_SIGEF__DIM__UNIDADE_GESTORA_GESTAO.parquet"
]

# Loop para criar as tabelas no DuckDB
for arquivo in tabelas:
    nome_tabela = os.path.splitext(arquivo)[0].lower()  # nome da tabela em minúsculo
    caminho = os.path.join(DATA_RAW, arquivo)
    con.execute(f"""
        CREATE TABLE IF NOT EXISTS {nome_tabela} AS
        SELECT * FROM '{caminho}'
    """)
    print(f"Tabela '{nome_tabela}' criada com sucesso!")

Tabela 'stg_sigef__fato__execucao_orcamentaria' criada com sucesso!
Tabela 'stg_sigef__dim__acao_programa' criada com sucesso!
Tabela 'stg_sigef__dim__conta_contabil' criada com sucesso!
Tabela 'stg_sigef__dim__credor' criada com sucesso!
Tabela 'stg_sigef__dim__domicilio_bancario' criada com sucesso!
Tabela 'stg_sigef__dim__evento' criada com sucesso!
Tabela 'stg_sigef__dim__fonte_recurso' criada com sucesso!
Tabela 'stg_sigef__dim__grupo_programacao_financeira' criada com sucesso!
Tabela 'stg_sigef__dim__natureza_despesa' criada com sucesso!
Tabela 'stg_sigef__dim__natureza_receita' criada com sucesso!
Tabela 'stg_sigef__dim__unidade_gestora_gestao' criada com sucesso!


In [7]:
df_est = con.execute("SELECT * FROM 'estimativa_cre_12-08-25' LIMIT 2").df()
df_est

,ug_codigo,descricao_ug,fonte_recurso,descricao_fonte,classificacao_ajustada,classificacao_intra,nomenclatura_receita,rec_grupo,rec_subgrupo,metodo_principal,ano,mes,valor
0,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125101.0,None,Imposto sobre a Propriedade de Veículos Automo...,Receita Bruta,Receita Estimada,IPCA + PIB,2025,1,65027941.98
1,140001,SECRETARIA DE ESTADO DE FINANCAS,500.0,Recursos não Vinculados de Impostos.,11125101.0,None,RENÚNCIA POTENCIAL - Imposto sobre a Proprieda...,Deduções da Receita,Renúncia Potencial,RENÚNCIA POTENCIAL,2025,1,0.00


In [8]:
df_est_500 = con.execute('SELECT mes, SUM(valor) FROM "estimativa_cre_12-08-25" WHERE fonte_recurso = 500 and ano = 2025 group by mes').df()
df_est_500

,mes,sum(valor)
0,1,8.729471e+08
1,2,9.823469e+08
2,3,8.355140e+08
3,4,8.609253e+08
4,5,9.494757e+08
5,6,1.007314e+09
6,7,8.794157e+08
7,8,8.799802e+08
8,9,7.929055e+08
9,10,8.157068e+08


In [19]:
df_fato = con.execute("SELECT * FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA LIMIT 2").df()
df_fato

,SIGEF_DB,MES_REFERENCIA,ID_UNIDADE_GESTORA_GESTAO,ID_FONTE_RECURSO,ID_NATUREZA_RECEITA,ID_NATUREZA_DESPESA,ID_ACAO_PROGRAMA,VALOR_RECEITA_PREVISTA_BRUTA,VALOR_RECEITA_PREVISTA_BRUTA_COTAS_MENSAIS,VALOR_RECEITA_PREVISTA_DEDUCOES,...,VALOR_DESPESA_CRONOGRAMA_DESEMBOLSO,VALOR_ALTERACOES_DESPESA_CREDITOS_ADICIONAIS,VALOR_ALTERACOES_DESPESA_REDUCAO_DESCENTRALIZACAO,VALOR_ALTERACOES_DESPESA_ACRESCIMO_DESCENTRALIZACAO,VALOR_DESPESA_CONTIGENCIAMENTO,VALOR_DESPESA_PREEMPENHOS,VALOR_DESPESA_OUTRAS_INDISPONIBILIDADES,VALOR_DESPESA_EMPENHADA,VALOR_DESPESA_LIQUIDADA,VALOR_DESPESA_PAGA
0,SIGEF2024,2024-03-01,66433312341ADD2DB089AEBB4624289E,05C1D02F5C8DAF887BA2E6C74F01BA3B,DCEEB932B4BCCF809D21389820688F3C,3ACF29FB703C251FCAFE3C310D517E81,E9F5B2E3A9588787EF8C451625997DA7,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1680744.15,1680744.15,1680744.15
1,SIGEF2024,2024-08-01,45BB2A98ADD87E0BC3335753EC8C26CE,13858FFA3BE74166EEAECDC7ABAD5093,DCEEB932B4BCCF809D21389820688F3C,69660F793F335D268005FB78FCBF86FB,F30B667192EB18932FBDC4C9BA278220,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-346260.01,0.00,0.00


In [10]:
df_fonte = con.execute("SELECT * FROM stg_sigef__dim__fonte_recurso LIMIT 2").df()
df_fonte

,SIGEF_DB,ID_FONTE_RECURSO,FONTE_REDUZIDO,DESCRICAO_FONTE_REDUZIDO,FONTE_RECURSO,DESCRICAO_FONTE_RECURSO,IDENTIFICADOR_USO,DESCRICAO_IDENTIFICADOR_USO,GRUPO_FONTE,DESCRICAO_GRUPO_FONTE,ESPECIFICACAO_FONTE,DESCRICAO_ESPECIFICACAO_FONTE,CLASSIFICACAO_FONTE,DESCRICAO_CLASSIFICACAO_FONTE
0,SIGEF2024,DF36ED34AC2446AB260DF91470A9EC1B,541,Transferências do FUNDEB - Complementação da U...,541001070,Ident. do perc. aplicado no pag. da remun. dos...,1,RECURSOS DO EXERCÍCIO CORRRENTE,5,DEMAIS VINCULAÇÕES DECORRENTES DE TRANSFERÊNCIAS,41,Recursos previdenciários,2,Recursos Vinculados à Educação
1,SIGEF2024,C50F3ED3EC72CE1D856B3B9DD605129C,542,Transferências do FUNDEB - Complementação da U...,542001070,Ident. do perc. aplicado no pag. da remun. dos...,1,RECURSOS DO EXERCÍCIO CORRRENTE,5,DEMAIS VINCULAÇÕES DECORRENTES DE TRANSFERÊNCIAS,42,Transferências do FUNDEB - Complementação da U...,2,Recursos Vinculados à Educação


In [41]:
# Consulta inicial (pode ser LIMIT 10 para preview)
df_duck = con.execute("""
    SELECT 
        EXTRACT(MONTH FROM a.MES_REFERENCIA) AS MES,
        SUM(a.VALOR_RECEITA_PREVISTA_BRUTA_COTAS_MENSAIS+a.VALOR_RECEITA_PREVISTA_DEDUCOES_COTAS_MENSAIS) AS RECEITA_PREVISTA,
        SUM(a.VALOR_RECEITA_REALIZADA_BRUTA+a.VALOR_RECEITA_REALIZADA_DEDUCOES) AS RECEITA_REALIZADA
    FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA a 
    JOIN stg_sigef__dim__fonte_recurso b 
        ON a.ID_FONTE_RECURSO = b.ID_FONTE_RECURSO
    WHERE b.FONTE_REDUZIDO = 500
        AND a.SIGEF_DB = 'SIGEF2025'
    GROUP BY a.MES_REFERENCIA
    ORDER BY EXTRACT(MONTH FROM a.MES_REFERENCIA)
""").df()

df_duck.head(13)

,MES,RECEITA_PREVISTA,RECEITA_REALIZADA
0,1,8.577561e+08,8.729471e+08
1,2,9.694977e+08,9.823469e+08
2,3,7.555875e+08,8.355140e+08
3,4,8.342721e+08,8.609253e+08
4,5,9.049794e+08,9.494757e+08
5,6,9.430365e+08,1.007314e+09
6,7,8.256935e+08,8.794157e+08
7,8,9.053202e+08,7.244658e+08
8,9,8.022829e+08,0.000000e+00
9,10,8.153100e+08,0.000000e+00


In [9]:
df_teste = con.execute("""
    SELECT 
            mes AS MES,
            SUM(valor) AS RECEITA_REALIZADA_EST
        FROM "estimativa_cre_12-08-25"
        WHERE fonte_recurso = 500
            AND ano = 2025
        GROUP BY mes
    """).df()

df_teste

,MES,RECEITA_REALIZADA_EST
0,1,8.729471e+08
1,2,9.823469e+08
2,3,8.355140e+08
3,4,8.609253e+08
4,5,9.494757e+08
5,6,1.007314e+09
6,7,8.794157e+08
7,8,8.799802e+08
8,9,7.929055e+08
9,10,8.157068e+08


In [11]:
df_final = con.execute("""
    WITH fato AS (
        SELECT 
            EXTRACT(MONTH FROM a.MES_REFERENCIA) AS MES,
            SUM(a.VALOR_RECEITA_PREVISTA_BRUTA_COTAS_MENSAIS + a.VALOR_RECEITA_PREVISTA_DEDUCOES_COTAS_MENSAIS) AS RECEITA_PREVISTA,
            SUM(a.VALOR_RECEITA_REALIZADA_BRUTA + a.VALOR_RECEITA_REALIZADA_DEDUCOES) AS RECEITA_REALIZADA_FATO
        FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA a
        JOIN stg_sigef__dim__fonte_recurso b
            ON a.ID_FONTE_RECURSO = b.ID_FONTE_RECURSO
        WHERE b.FONTE_REDUZIDO = 500
            AND a.SIGEF_DB = 'SIGEF2025'
        GROUP BY EXTRACT(MONTH FROM a.MES_REFERENCIA)
    ),
    est AS (
        SELECT 
            mes AS MES,
            SUM(valor) AS RECEITA_REALIZADA_EST
        FROM "estimativa_cre_12-08-25"
        WHERE fonte_recurso = 500
            AND ano = 2025
        GROUP BY mes
    )
    SELECT
        f.MES,
        f.RECEITA_PREVISTA,
        CASE 
            WHEN f.MES >= 8 THEN e.RECEITA_REALIZADA_EST
            ELSE f.RECEITA_REALIZADA_FATO
        END AS RECEITA_REALIZADA,
        ((RECEITA_REALIZADA - RECEITA_PREVISTA) / RECEITA_PREVISTA) * 100 AS PERCENTUAL_REALIZADO,
        (RECEITA_REALIZADA - RECEITA_PREVISTA) AS DIFERENCA_VALOR
    FROM fato f
    LEFT JOIN est e
        ON f.MES = e.MES
    ORDER BY f.MES
""").df()


print("Tabela 2 - Previsão Atualizada x Receita Arrecadada/Projetada 2025 - Fonte 500")
df_final

Tabela 2 - Previsão Atualizada x Receita Arrecadada/Projetada 2025 - Fonte 500


,MES,RECEITA_PREVISTA,RECEITA_REALIZADA,PERCENTUAL_REALIZADO,DIFERENCA_VALOR
0,1,8.577561e+08,8.729471e+08,1.771013,1.519097e+07
1,2,9.694977e+08,9.823469e+08,1.325339,1.284913e+07
2,3,7.555875e+08,8.355140e+08,10.578060,7.992649e+07
3,4,8.342721e+08,8.609253e+08,3.194786,2.665321e+07
4,5,9.049794e+08,9.494757e+08,4.916834,4.449633e+07
5,6,9.430365e+08,1.007314e+09,6.816005,6.427742e+07
6,7,8.256935e+08,8.794157e+08,6.506313,5.372221e+07
7,8,9.053202e+08,8.799802e+08,-2.799009,-2.533999e+07
8,9,8.022829e+08,7.929055e+08,-1.168833,-9.377346e+06
9,10,8.153100e+08,8.157068e+08,0.048676,3.968614e+05


In [17]:
df_final = con.execute("""
    WITH fato AS (
        SELECT 
            EXTRACT(MONTH FROM a.MES_REFERENCIA) AS MES,
            SUM(a.VALOR_RECEITA_PREVISTA_BRUTA_COTAS_MENSAIS + a.VALOR_RECEITA_PREVISTA_DEDUCOES_COTAS_MENSAIS) AS RECEITA_PREVISTA,
            SUM(a.VALOR_RECEITA_REALIZADA_BRUTA + a.VALOR_RECEITA_REALIZADA_DEDUCOES) AS RECEITA_REALIZADA_FATO
        FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA a
        JOIN stg_sigef__dim__fonte_recurso b
            ON a.ID_FONTE_RECURSO = b.ID_FONTE_RECURSO
        WHERE b.FONTE_REDUZIDO = 500
            AND a.SIGEF_DB = 'SIGEF2025'
        GROUP BY EXTRACT(MONTH FROM a.MES_REFERENCIA)
    ),
    est AS (
        SELECT 
            mes AS MES,
            SUM(valor) AS RECEITA_REALIZADA_EST
        FROM "estimativa_cre_12-08-25"
        WHERE fonte_recurso = 500
            AND ano = 2025
        GROUP BY mes
    )
    SELECT
        f.MES,
        (f.RECEITA_PREVISTA) * .7495 AS RECEITA_PREVISTA,
        (CASE 
            WHEN f.MES >= 8 THEN e.RECEITA_REALIZADA_EST
            ELSE f.RECEITA_REALIZADA_FATO
        END) * .7495 AS RECEITA_REALIZADA,
        ((RECEITA_REALIZADA - RECEITA_PREVISTA) / RECEITA_PREVISTA) * 100 AS PERCENTUAL_REALIZADO,
        (RECEITA_REALIZADA - RECEITA_PREVISTA) AS DIFERENCA_VALOR
    FROM fato f
    LEFT JOIN est e
        ON f.MES = e.MES
    ORDER BY f.MES
""").df()


print("Tabela 3 - Previsão Atualizada x Receita Arrecadada/Projetada 2025 - Fonte 500 - Poder Executivo líquido duodécimos")
df_final

Tabela 3 - Previsão Atualizada x Receita Arrecadada/Projetada 2025 - Fonte 500 - Poder Executivo líquido duodécimos


,MES,RECEITA_PREVISTA,RECEITA_REALIZADA,PERCENTUAL_REALIZADO,DIFERENCA_VALOR
0,1,6.428882e+08,6.542738e+08,-23.722626,-2.034823e+08
1,2,7.266385e+08,7.362690e+08,-24.056658,-2.332288e+08
2,3,5.663128e+08,6.262177e+08,-17.121744,-1.293698e+08
3,4,6.252869e+08,6.452635e+08,-22.655508,-1.890086e+08
4,5,6.782820e+08,7.116320e+08,-21.364833,-1.933473e+08
5,6,7.068059e+08,7.549818e+08,-19.941404,-1.880547e+08
6,7,6.188573e+08,6.591221e+08,-20.173518,-1.665714e+08
7,8,6.785375e+08,6.595452e+08,-27.147857,-2.457750e+08
8,9,6.013110e+08,5.942827e+08,-25.926040,-2.080002e+08
9,10,6.110748e+08,6.113723e+08,-25.013517,-2.039377e+08


In [18]:
from pandasgui import show

gui = show(df_final)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\Acer\AppData\Local\Temp\ipykernel_4528\2774868566.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gui = show(df_final)
C:\Users\Acer\AppData\Local\Temp\ipykernel_4528\2774868566.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gui = show(df_final)
C:\Users\Acer\AppData\Local\Temp\ipykernel_4528\2774868566.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\A

C:\Users\Acer\AppData\Roaming\Python\Python311\site-packages\ipykernel\eventloops.py:128: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Acer\AppData\Roaming\Python\Python311\site-packages\ipykernel\eventloops.py:128: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Acer\AppData\Roaming\Python\Python311\site-packages\ipykernel\eventloops.py:128: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Acer\AppData\Roamin

Refreshed _17


In [24]:
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.getenv("HF_TOKEN"),
)

def consulta_ia(pergunta: str):
    prompt = f"""
    Você é um assistente que gera consultas SQL para DuckDB.
    A tabela disponível se chama 'estimativa_cre' e tem as seguintes colunas:
    {', '.join(df_duck.columns.tolist())}

    Pergunta do usuário: {pergunta}

    Responda apenas com a SQL.
    """

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0,
    )

    query = response.choices[0].message.content.strip()

    # Executa a query no DuckDB
    try:
        resultado = con.execute(query).fetchdf()
        return query, resultado
    except Exception as e:
        return query, f"Erro ao executar SQL: {e}"

In [ ]:
query, resultado = consulta_ia("O total arrecadado por ano da fonte 500")
print("Query usada:\n", query)
print("\nResultado:\n", resultado)